In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict
from PIL import Image
from scipy import ndimage
import pandas as pd
import imageio
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers

from datetime import datetime
from packaging import version

%matplotlib inline

Using TensorFlow backend.


In [21]:
def convert_to_indices(arr, binsPower):
    arr = np.around(arr, decimals=binsPower)
    arr = np.multiply(arr, 10**(binsPower))
    return arr.astype(int)

def onehot_initialization(a, binsPower):
    ncols = 10 **(binsPower) + 1
    out = np.zeros(a.shape + (ncols,), dtype=int)
    out[all_idx(a, axis=2)] = 1
    return out

def all_idx(idx, axis):
    grid = np.ogrid[tuple(map(slice, idx.shape))]
    grid.insert(axis, idx)
    return tuple(grid)

def convert_onehot(arr, binPower):
    arr = convert_to_indices(arr, binPower)
    arr = onehot_initialization(arr, binPower)
    return arr

In [22]:
# labels = np.genfromtxt('labels.csv', delimiter=',')
labels = pd.read_csv('labels.csv', float_precision='road_trip')
labels.head(5)

,Unnamed: 0,img_name,base,base color R,base color G,base color B,specular,specular color R,specular color G,specular color B,specular roughness,metalness
0,0,0.bmp,0.196696,[0.02551167982996516,0.523569,0.09533690194549593],0.903675,[0.9414576345001383,0.927580,0.418087362819755],0.120439,0.941755
1,1,1.bmp,0.563387,[0.13560656102894886,0.715835,0.6311335628961977],0.381407,[0.9228773407900674,0.328530,0.294482323738513],0.439152,0.511734
2,2,2.bmp,0.433630,[0.06292513279208145,0.468129,0.49781547380782],0.622876,[0.8260393377700712,0.115993,0.1549741111822439],0.646591,0.490058
3,3,3.bmp,0.471443,[0.8909351516333082,0.133360,0.7187711144029106],0.419356,[0.08579008506320773,0.466882,0.36951439970192435],0.154928,0.632136
4,4,4.bmp,0.172493,[0.1089380911674539,0.179026,0.4079338804909298],0.893968,[0.9620524644071147,0.906355,0.3523653506316685],0.708425,0.002353


In [23]:
labels["base color R"] = labels["base color R"].str[1:]
labels["base color B"] = labels["base color B"].str[:-1]
labels["specular color R"] = labels["specular color R"].str[1:]
labels["specular color B"] = labels["specular color B"].str[:-1]
labels["img_name"] = labels["img_name"].str[:-3]+"jpg"

In [24]:
labels.head(5)

,Unnamed: 0,img_name,base,base color R,base color G,base color B,specular,specular color R,specular color G,specular color B,specular roughness,metalness
0,0,0.jpg,0.196696,0.02551167982996516,0.523569,0.09533690194549593,0.903675,0.9414576345001383,0.927580,0.418087362819755,0.120439,0.941755
1,1,1.jpg,0.563387,0.13560656102894886,0.715835,0.6311335628961977,0.381407,0.9228773407900674,0.328530,0.294482323738513,0.439152,0.511734
2,2,2.jpg,0.433630,0.06292513279208145,0.468129,0.49781547380782,0.622876,0.8260393377700712,0.115993,0.1549741111822439,0.646591,0.490058
3,3,3.jpg,0.471443,0.8909351516333082,0.133360,0.7187711144029106,0.419356,0.08579008506320773,0.466882,0.36951439970192435,0.154928,0.632136
4,4,4.jpg,0.172493,0.1089380911674539,0.179026,0.4079338804909298,0.893968,0.9620524644071147,0.906355,0.3523653506316685,0.708425,0.002353


In [25]:
vector_values = labels.iloc[:,2:]
vector_values = vector_values.astype(float)

In [26]:
np_labels = vector_values.values
np_labels[0]

array([0.19669609, 0.02551168, 0.52356934, 0.0953369 , 0.90367509,
       0.94145763, 0.92757973, 0.41808736, 0.1204395 , 0.94175506])

In [33]:
out = convert_onehot(np_labels, 2)
out[0,2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
# rest of 6000 images
labels_rest = pd.read_csv('big_dataset.csv', float_precision='road_trip')
labels_rest["img_name"] = labels_rest["img_name"].str[:-3]+"jpg"
labels_rest.head(5)

FileNotFoundError: File b'big_dataset.csv' does not exist

In [ ]:
vector_values2 = labels_rest.iloc[:,2:]
vector_values2 = vector_values2.astype(float)

In [ ]:
# concat the 4000 and 6000 images for full dataset
frames = [vector_values, vector_values2]
vector_value = pd.concat(frames)

In [ ]:
np_labels = vector_value.values
np_labels = convert_onehot(np_labels, 3)

THE BELOW CELL TAKES AGES TO RUN. ONLY RUN IT ONCE.

In [94]:
# check to make sure it works. should equal number of categories (10)
print(np.sum(np_labels[0]))

10


In [95]:
print(np_labels.shape)

(10000, 10, 1001)
